In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive 
import scipy.stats as ss
import imageio

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# This dataset has been acquired from https://www.kaggle.com/datasets/pralabhpoudel/world-energy-consumption
Energy=pd.read_csv('gdrive/MyDrive/Colab Notebooks/World Energy Consumption.csv')

In [4]:
#List begins with columns that will clearly be needed
nuclear_cols = ["year","country","carbon_intensity_elec"]
# And then it will be appended with all columns related to nuclear
for col in Energy.columns:
  if "nuclear" in col:
    nuclear_cols.append(col)

In [5]:
nuclear_cols.append("primary_energy_consumption")
#The analisys will comprise just the 21st century
Nuclear = Energy.loc[Energy["year"]>=2001,nuclear_cols].dropna(how = "all")

In [6]:
# Only countries which produce some nuclear energy will be taken into account
Nuclear = Nuclear.loc[Nuclear["nuclear_electricity"]>0,:]

In [7]:
plt.figure(figsize = ((8,8)))
countries = Nuclear.country.value_counts().index
lr = []
images = []
"""
For every year a linear regression is performed:
--> X: Share of nuclear energy production
--> Y: Carbon Intensity 
"""
for ye in range(2001,2020):
  # Linear regression
  plt.figure(figsize = ((8,8)))
  # Select info for linear regression
  forlr = Nuclear.loc[Nuclear["year"] == ye, ["country","nuclear_share_elec","carbon_intensity_elec"]].set_index("country")
  forlr = forlr.loc[forlr["carbon_intensity_elec"].notna()]

  # Linear regression paramters and indicators are stored in a list.
  lr.append(ss.linregress(x = forlr.nuclear_share_elec, y = forlr.carbon_intensity_elec))
  
  # Scatterplot of points the liner regressions have been performed with. The more consumption the country presents, the bigger its marker is.
  g = sns.scatterplot(x = "nuclear_share_elec", y = "carbon_intensity_elec", data = Nuclear.loc[Nuclear["year"] == ye,:],
                  size = Nuclear.loc[Nuclear["year"] == ye,"primary_energy_consumption"], 
                  sizes = (1e2, 1e5), palette = "pastel", legend = False)
  
  # Limits are fixed for all year for better observetion
  g.set(xlim=(0, 85), ylim=(0, 700))

  # Write names of countries next to their respective markers
  for c in forlr.index:  
    g.annotate(c, xy = (forlr.loc[c,"nuclear_share_elec"], forlr.loc[c,"carbon_intensity_elec"]))

  # Plot linear regressions
  sns.lineplot(x = forlr.nuclear_share_elec, y = lr[-1].intercept + lr[-1].slope*forlr.nuclear_share_elec, color = "red")
  plt.xlabel("Nuclear Power Share for electricity production [%]")
  plt.ylabel("Carbon intensity of electricity production, [g(CO$_2$) / kWh]")
  
  # Write current year in the bottom left corner
  g.annotate(str(ye), xy = (2,20), fontsize = 24)

  # Each figure is saved
  plt.savefig(f'gdrive/MyDrive/Colab Notebooks/img/img_{ye}.png', 
                transparent = False)
  plt.close()  

<Figure size 576x576 with 0 Axes>

In [8]:
# A list of images is created so a gif can be obtained
for ye in range(2001,2020):
  im = imageio.imread(f'gdrive/MyDrive/Colab Notebooks/img/img_{ye}.png')
  images.append(im)

In [9]:
# Save gif
imageio.mimsave('gdrive/MyDrive/Colab Notebooks/img/nuc_carb.gif', # output gif
                images,          # array of input frames
                fps = 1)         # optional: frames per second

In [11]:
# See pvalue for all years' regressions.
for r,y in zip(lr,range(2001,2020)):
  print("pvalue for year {}: {}".format(y, r.pvalue))

pvalue for year 2001: 0.0019289094057463236
pvalue for year 2002: 0.0012286365486791288
pvalue for year 2003: 0.0003764037412933048
pvalue for year 2004: 0.0010559789959719906
pvalue for year 2005: 0.004037780348932975
pvalue for year 2006: 0.001904325533791953
pvalue for year 2007: 0.0010826528414018992
pvalue for year 2008: 0.002919188157621511
pvalue for year 2009: 0.00489522950439222
pvalue for year 2010: 0.015643179840147216
pvalue for year 2011: 0.009442289561110633
pvalue for year 2012: 0.00787788524989414
pvalue for year 2013: 0.003857821020893346
pvalue for year 2014: 0.004139034589839585
pvalue for year 2015: 0.007557443054234676
pvalue for year 2016: 0.006722541061182497
pvalue for year 2017: 0.012163133049944345
pvalue for year 2018: 0.011344948918448025
pvalue for year 2019: 0.034465000976257196
